# Transfer Learning for Pretrained Models

### 1. Import dependencies

In [1]:
import os
import pandas as pd

import torch
from torch.utils.data import DataLoader
from torchvision import transforms as T
from PIL import Image
from typing import List

# from datasets import load_dataset

/home/oem/miniconda3/envs/skin_cancer/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2.1 Load dataset (Hugging Face)

Please choose to either load from 2.1 hugging face or 2.2 from file dataset was previously uploaded to huggingface.

In [2]:
# dataset = load_dataset("marmal88/skin_cancer")
# dataset.set_format(type='torch', columns=['image', 'image_id', 'lesion_id', 'dx', 'dx_type', 'age', 'sex', 'localization'])

# # print to get the names of the sets within the dataset
# print(dataset.__getitem__)

### 2.2 Load dataset from file

In [3]:
# insert your own root directory
path_to_root = "/home/oem/Documents/coding/personal/computer_vision_toolkit"
os.chdir(path_to_root)
os.getcwd()

'/home/oem/Documents/coding/personal/skin_cancer'

In [4]:
df = pd.read_csv(os.path.join(os.getcwd(), "notebooks/dataset.csv"))
df.head(10)

,file_name,image_id,lesion_id,dx,dx_type,age,sex,localization,label
0,data/ham10000_images_part_2/ISIC_0031774.jpg,ISIC_0031774,HAM_0002275,melanocytic_Nevi,follow_up,45.0,female,lower extremity,0
1,data/ham10000_images_part_2/ISIC_0030527.jpg,ISIC_0030527,HAM_0006713,melanocytic_Nevi,follow_up,50.0,female,trunk,0
2,data/ham10000_images_part_2/ISIC_0033561.jpg,ISIC_0033561,HAM_0004708,melanocytic_Nevi,histo,45.0,male,trunk,0
3,data/ham10000_images_part_2/ISIC_0034041.jpg,ISIC_0034041,HAM_0005496,melanocytic_Nevi,histo,15.0,female,lower extremity,0
4,data/ham10000_images_part_2/ISIC_0031369.jpg,ISIC_0031369,HAM_0000531,melanoma,histo,85.0,male,face,1
5,data/ham10000_images_part_2/ISIC_0033179.jpg,ISIC_0033179,HAM_0002898,melanocytic_Nevi,histo,80.0,female,upper extremity,0
6,data/ham10000_images_part_2/ISIC_0030478.jpg,ISIC_0030478,HAM_0000894,melanocytic_Nevi,follow_up,45.0,female,upper extremity,0
7,data/ham10000_images_part_2/ISIC_0029344.jpg,ISIC_0029344,HAM_0004638,melanocytic_Nevi,histo,45.0,female,back,0
8,data/ham10000_images_part_2/ISIC_0030824.jpg,ISIC_0030824,HAM_0000330,melanoma,histo,60.0,male,chest,1
9,data/ham10000_images_part_2/ISIC_0032773.jpg,ISIC_0032773,HAM_0000571,benign_keratosis-like_lesions,consensus,70.0,male,back,2


In [5]:
# transforming the labels to numeric
lesion_type_dict = {
    'melanocytic_Nevi': 0,
    'melanoma': 1,
    'benign_keratosis-like_lesions': 2,
    'basal_cell_carcinoma': 3,
    'actinic_keratoses': 4,
    'vascular_lesions': 5,
    'dermatofibroma': 6
}

df['label'] = df.dx.map(lesion_type_dict)
df['label'] = df["label"].astype(int)
df.to_csv("notebooks/dataset.csv", index=False)

In [6]:
df.head()

,file_name,image_id,lesion_id,dx,dx_type,age,sex,localization,label
0,data/ham10000_images_part_2/ISIC_0031774.jpg,ISIC_0031774,HAM_0002275,melanocytic_Nevi,follow_up,45.0,female,lower extremity,0
1,data/ham10000_images_part_2/ISIC_0030527.jpg,ISIC_0030527,HAM_0006713,melanocytic_Nevi,follow_up,50.0,female,trunk,0
2,data/ham10000_images_part_2/ISIC_0033561.jpg,ISIC_0033561,HAM_0004708,melanocytic_Nevi,histo,45.0,male,trunk,0
3,data/ham10000_images_part_2/ISIC_0034041.jpg,ISIC_0034041,HAM_0005496,melanocytic_Nevi,histo,15.0,female,lower extremity,0
4,data/ham10000_images_part_2/ISIC_0031369.jpg,ISIC_0031369,HAM_0000531,melanoma,histo,85.0,male,face,1


In [7]:
from src.dataset import CustomImageDataset 

In [8]:
# mean and standard dev as per pre-trained imagenet dataset (https://pytorch.org/hub/pytorch_vision_resnet/)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = T.Compose([
            T.Resize([224,224]),
            T.ToTensor(),
            # normalize mean and std from imagenet pretrained
            T.Normalize(mean=mean, std=std),
            ])

dataset = CustomImageDataset("notebooks/dataset.csv", transform)

### 3. Transform original 

Split and transform the dataset into train, validation and test sets

In [9]:
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [8000, 2000, 15])

In [10]:
# load the respective train validation and test split to dataloader
train_loader = DataLoader(train_set, batch_size=5, shuffle=True)
val_loader = DataLoader(val_set, batch_size=5, shuffle=True)
test_loader = DataLoader(test_set, batch_size=5, shuffle=True)

### 4. Model training


In [11]:
import torch.nn as nn
from torch.optim import Adam
import torchvision 
from torchvision.models import resnet50, ResNet50_Weights

In [12]:
num_class = df.label.nunique()
learning_rate = 0.001
epochs = 5

In [13]:
class ClassifierModel(nn.Module):
    """ The classic transfer learning template where the models can be swapped out
        for other types depending on senario 
    """    
    def __init__(self, num_class:int):
        """ Initializes the ClassifierModel instance
            The super here inherits the functions from the base torch nn.Module, allowing 
            us to create layers and convolutions.
            Added a dropout to the last linear layer and amended out_features to num_class
        Args:
            num_class (int): The number of classes in the classification problem.
        """        
        super().__init__()
        self.model = torchvision.models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
                            nn.Dropout(0.5), 
                            nn.Linear(num_ftrs, num_class)
                            )

    def forward(self, x:torch.Tensor)-> torch.Tensor:
        x = self.model(x)
        return x

In [14]:

model = ClassifierModel(num_class)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate) 

In [15]:
for epoch in range(epochs):
    print(f"current epoch {epoch}")
    for img, label in train_loader:
        optimizer.zero_grad()
        output = model(img)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()

with torch.no_grad():
    correct = 0
    total = 0
    for img, label in train_loader:
        print("Evaluation loop starts")
        model.eval()
        output = model(img)
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    print(f"Accuracy of the model on the test set: {100 * correct / total}%")

current epoch 0


## 5. Other learning points

As torch is written in a pythonic way, amending layers is a relatively straightforward affair.  

Below is a non-exhaustive set of operations you can do:  
1. Identifying layers in torch pretrained models
2. Swapping out layers in torch pretrained models
3. Adding new layers

### 1. Identifying layers in torch pretrained models

We can identify the model layer name 
For example, we can modify the fully connected layer (fc) by getting out the attribute names:
1. model.fc.in_features - gives us the shape of the tensor going in
2. model.fc.out_features - can be amended from 1000 (1000 objects from imagenet) to the number of classes in our own dataset.

In [1]:
import torchvision
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn
from torch.nn import functional as F

# You can
m = torchvision.models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)


print(f"Here we can see that the input is: {m.fc.in_features} and the output is {m.fc.out_features}")
print(m.fc)

/home/oem/miniconda3/envs/skin_cancer/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here we can see that the input is: 2048 and the output is 1000
Linear(in_features=2048, out_features=1000, bias=True)


### 2. Swapping out layers in torch pretrained models

Since we can identify each layer by its name, similarly we can create our own layers (newlayer) and replace them at the corresponding point.

In [2]:
# Create a new sequential layer 
newlayer = nn.Sequential(
                nn.Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False),
                nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )

# Assign newly created layer to existing layer
m.layer2 = newlayer
m.layer2

Sequential(
  (0): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

### 3. Adding new layers

New layers can also be added to already existing architectures. For example, we can take the outputs from one layer and add it to new layers, this is different from modifying the last layer, though in some cases it might be easier to implement it this way.

Please also see this article on adding layers vs replacing layers [here](https://discuss.pytorch.org/t/how-to-add-a-layer-to-an-existing-neural-network/30129/4)

In [3]:
# taking the fully connected out_features and adding it to the new sequential
newlayer = nn.Conv2d(m.fc.out_features, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)

new_model = nn.Sequential(m, newlayer)
new_model

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

### Note of Caution

You might be tempted to edit the model after instantiating the object. However, I think its cleaner, clearer (and perhaps less buggy), if the models were modified during instantiation as per below.

In [4]:
class MyModel(nn.Module):
    def __init__(self, in_features, nb_classes, nb_hidden_layer, 
        hidden_size, act=nn.ReLU):
        super(MyModel, self).__init__()
        self.act = act()
        
        self.fc1 = nn.Linear(in_features, hidden_size)
        self.fcs = nn.ModuleList([nn.Linear(hidden_size, hidden_size)])
        self.out = nn.Linear(hidden_size, nb_classes)
        
    def forward(self, x):
        x = self.act(self.fc1(x))
        for l in self.fcs:
            x = F.relu(l(x))
        x = self.out(x)
        return x
            
model = MyModel(2, 3, 4, 5, nn.ReLU)